# M1M3 LUT Iterative Improvement Results Notebook

This notebook generates the plots that show how the LUT and the applied balance forces have changed over multiple LUT iterations.

Relevant imports

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import warnings

from dataclasses import dataclass
from numpy.polynomial import Polynomial
from astropy.time import Time, TimeDelta

warnings.filterwarnings('ignore')

from lsst_efd_client import EfdClient
from lsst.ts.xml.tables.m1m3 import FATable
from lsst.sitcom import vandv

Example code of how to plot fields in different EFD topics.

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
async def get_balance_forces(start, end, resample_rate='1T'):
    # Retrieve elevations
    elevations = await client.select_time_series(
        'lsst.sal.MTMount.elevation',
        ['actualPosition', 'timestamp'],  
        start, 
        end,
    )  
    elevations = elevations['actualPosition'].resample('1T').mean()

    forces_z = await client.select_time_series(
        "lsst.sal.MTM1M3.appliedBalanceForces", 
        zForce, 
        start,
        end
    )

    forces_y = await client.select_time_series(
        "lsst.sal.MTM1M3.appliedBalanceForces", 
        yForce, 
        start,
        end
    )

    # join dataframes and resample them.
    forces = pd.concat([forces_y.dropna(axis = 1), forces_z.dropna(axis = 1)])
    forces = forces.resample('1T').mean()
    
    return forces, elevations

async def get_hardpoint_forces(start, end, resample_rate='1T'):
    # Retrieve elevations
    elevations = await client.select_time_series(
        'lsst.sal.MTMount.elevation',
        ['actualPosition', 'timestamp'],  
        start, 
        end,
    )  
    elevations = elevations['actualPosition'].resample('1T').mean()

    forces = await client.select_time_series(
        "lsst.sal.MTM1M3.hardpointActuatorData", 
        ['fx', 'fy', 'fz', 'mx', 'my', 'mz'], 
        start,
        end
    )
    
    forces = forces.resample('1T').mean()
    
    return forces, elevations

async def get_hardpoint_forces_in(start, end, resample_rate='1T'):
    # Retrieve elevations
    elevations = await client.select_time_series(
        'lsst.sal.MTMount.elevation',
        ['actualPosition', 'timestamp'],  
        start, 
        end,
    )  
    elevations = elevations['actualPosition'].resample('1T').mean()

    forces = await client.select_time_series(
        "lsst.sal.MTM1M3.hardpointActuatorData", 
        ['measuredForce0', 'measuredForce1', 'measuredForce2', 'measuredForce3', 'measuredForce4','measuredForce5'], 
        start,
        end
    )
    
    forces = forces.resample('1T').mean()
    
    return forces, elevations


Set start and end time of the sequence that we want to analyze

Create ids vector

In [ ]:
ids = np.array([fa.actuator_id for fa in FATable])

xForce = [str("".join(("xForces",str(i)))) for i in range(12)]
yForce = [str("".join(("yForces",str(i)))) for i in range(100)]
zForce = [str("".join(("zForces",str(i)))) for i in range(156)]

## Plot balance forces

In [ ]:
start = Time('2023-05-31 08:35:0Z', scale='utc')
end = Time('2023-05-31 09:40:0Z', scale='utc')

forces_it0, elevations_it0 = await get_balance_forces(start, end)

In [ ]:
start = Time('2023-06-16 02:30:0Z', scale='utc')
end = Time('2023-06-16 03:01:0Z', scale='utc')

forces_it1, elevations_it1 = await get_balance_forces(start, end)

In [ ]:
start = Time('2023-06-21 05:35:0Z', scale='utc')
end = Time('2023-06-21 06:26:0Z', scale='utc')

forces_it2, elevations_it2 = await get_balance_forces(start, end)

In [ ]:
start = Time('2023-06-22 02:20:0Z', scale='utc')
end = Time('2023-06-22 03:45:0Z', scale='utc')

forces_it3, elevations_it3 = await get_balance_forces(start, end)

In [ ]:
start = Time('2023-06-22 21:38:0Z', scale='utc')
end = Time('2023-06-22 23:16:0Z', scale='utc')

forces_it4, elevations_it4 = await get_balance_forces(start, end)

In [ ]:
start = Time('2023-06-24 02:13:0Z', scale='utc')
end = Time('2023-06-24 03:01:0Z', scale='utc')

forces_it5, elevations_it5 = await get_balance_forces(start, end)

In [ ]:
start = Time('2023-06-27 01:35:0Z', scale='utc')
end = Time('2023-06-27 02:43:5Z', scale='utc')

forces_it6, elevations_it6 = await get_balance_forces(start, end)

In [ ]:
start = Time('2023-06-27 11:45:0Z', scale='utc')
end = Time('2023-06-27 12:12:0Z', scale='utc')

forces_it7, elevations_it7 = await get_balance_forces(start, end)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize = (26,120))
for idx in range(10):
    plt.subplot(40,7,7*idx + 1)
    plt.plot(90 - elevations_it0, forces_it0[zForce[idx]], '-', label = 'Balance forces')
    plt.title(f'{zForce[idx]} - iteration 0')
    plt.ylabel('Balance Force in Z (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(40,7,7*idx + 2)
    plt.plot(90 - elevations_it1, forces_it1[zForce[idx]], '-', label = 'Balance forces')
    plt.title(f'{zForce[idx]} - iteration 1')
    plt.ylabel('Balance Force in Z (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(40,7,7*idx + 3)
    plt.plot(90 - elevations_it2, forces_it2[zForce[idx]], '-', label = 'Balance forces')
    plt.title(f'{zForce[idx]} - iteration 2')
    plt.ylabel('Balance Force in Z (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(40,7,7*idx + 4)
    plt.plot(90 - elevations_it3, forces_it3[zForce[idx]], '-', label = 'Balance forces')
    plt.title(f'{zForce[idx]} - iteration 3')
    plt.ylabel('Balance Force in Z (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(40,7,7*idx + 5)
    plt.plot(90 - elevations_it4, forces_it4[zForce[idx]], '-', label = 'Balance forces')
    plt.title(f'{zForce[idx]} - iteration 4')
    plt.ylabel('Balance Force in Z (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(40,7,7*idx + 6)
    plt.plot(90 - elevations_it5, forces_it5[zForce[idx]], '-', label = 'Balance forces')
    plt.title(f'{zForce[idx]} - iteration 5')
    plt.ylabel('Balance Force in Z (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(40,7,7*idx + 7)
    plt.plot(90 - elevations_it7, forces_it7[zForce[idx]], '-', label = 'Balance forces')
    plt.title(f'{zForce[idx]} - iteration 7')
    plt.ylabel('Balance Force in Z (N)')
    plt.xlabel('Zenith (deg)')
    
plt.tight_layout()

## Plot total measured forces

In [ ]:
start = Time('2023-05-29 23:16:0Z', scale='utc')
end = Time('2023-05-29 23:47:0Z', scale='utc')

forces_hp_it0, elevations_hp_it0 = await get_hardpoint_forces(start, end)
forces_hp_in_it0, elevations_hp_in_it0 = await get_hardpoint_forces_in(start, end)

In [ ]:
start = Time('2023-06-16 03:13:0Z', scale='utc')
end = Time('2023-06-16 03:49:0Z', scale='utc')

forces_hp_it1, elevations_hp_it1 = await get_hardpoint_forces(start, end)
forces_hp_in_it1, elevations_hp_in_it1 = await get_hardpoint_forces_in(start, end)

In [ ]:
start = Time('2023-06-22 04:08:0Z', scale='utc')
end = Time('2023-06-22 04:49:0Z', scale='utc')

forces_hp_it3, elevations_hp_it3 = await get_hardpoint_forces(start, end)
forces_hp_in_it3, elevations_hp_in_it3 = await get_hardpoint_forces_in(start, end)

In [ ]:
start = Time('2023-06-27 00:13:0Z', scale='utc')
end = Time('2023-06-27 01:22:5Z', scale='utc')

forces_hp_it6, elevations_hp_it6 = await get_hardpoint_forces(start, end)
forces_hp_in_it6, elevations_hp_in_it6 = await get_hardpoint_forces_in(start, end)

In [ ]:
start = Time('2023-06-27 08:06:0Z', scale='utc')
end = Time('2023-06-27 08:40:0Z', scale='utc')

forces_hp_it7, elevations_hp_it7 = await get_hardpoint_forces(start, end)
forces_hp_in_it7, elevations_hp_in_it7 = await get_hardpoint_forces_in(start, end)

In [ ]:
%matplotlib inline
hpForces = ['fx', 'fy', 'fz', 'mx', 'my', 'mz']
fig = plt.figure(figsize = (15,17))
for idx in range(6):
    plt.subplot(6,5,5*idx + 1)
    plt.plot(90 - elevations_hp_it0, forces_hp_it0[hpForces[idx]], '-', label = 'Balance forces')
    plt.title(f'{hpForces[idx]} - iteration 0')
    plt.ylabel(f'{hpForces[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 2)
    plt.plot(90 - elevations_hp_it1, forces_hp_it1[hpForces[idx]], '-', label = 'Balance forces')
    plt.title(f'{hpForces[idx]} - iteration 1')
    plt.ylabel(f'{hpForces[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 3)
    plt.plot(90 - elevations_hp_it3, forces_hp_it3[hpForces[idx]], '-', label = 'Balance forces')
    plt.title(f'{hpForces[idx]} - iteration 3')
    plt.ylabel(f'{hpForces[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 4)
    plt.plot(90 - elevations_hp_it6, forces_hp_it6[hpForces[idx]], '-', label = 'Balance forces')
    plt.title(f'{hpForces[idx]} - iteration 6')
    plt.ylabel(f'{hpForces[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 5)
    plt.plot(90 - elevations_hp_it7, forces_hp_it7[hpForces[idx]], '-', label = 'Balance forces')
    plt.title(f'{hpForces[idx]} - iteration 7')
    plt.ylabel(f'{hpForces[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
plt.tight_layout()
plt.savefig('Evolution_hardpoint_forces.png')

## Plot total hardpoint forces

In [ ]:
hps = ['HP1', 'HP2', 'HP3', 'HP4', 'HP5', 'HP6']
hpsm = ['measuredForce0', 'measuredForce1', 'measuredForce2', 'measuredForce3', 'measuredForce4','measuredForce5']

fig = plt.figure(figsize = (15, 17))
for idx in range(6):
    plt.subplot(6,5,5*idx + 1)
    plt.plot(90 - elevations_hp_in_it0, forces_hp_in_it0[hpsm[idx]], '-', label = 'Balance forces')
    plt.title(f'{hps[idx]} - iteration 0')
    plt.ylabel(f'{hps[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 2)
    plt.plot(90 - elevations_hp_in_it1, forces_hp_in_it1[hpsm[idx]], '-', label = 'Balance forces')
    plt.title(f'{hps[idx]} - iteration 1')
    plt.ylabel(f'{hps[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 3)
    plt.plot(90 - elevations_hp_in_it3, forces_hp_in_it3[hpsm[idx]], '-', label = 'Balance forces')
    plt.title(f'{hps[idx]} - iteration 3')
    plt.ylabel(f'{hps[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 4)
    plt.plot(90 - elevations_hp_in_it6, forces_hp_in_it6[hpsm[idx]], '-', label = 'Balance forces')
    plt.title(f'{hps[idx]} - iteration 6')
    plt.ylabel(f'{hps[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
    plt.subplot(6,5,5*idx + 5)
    plt.plot(90 - elevations_hp_in_it7[2:], forces_hp_in_it7[hpsm[idx]][2:], '-', label = 'Balance forces')
    plt.title(f'{hps[idx]} - iteration 7')
    plt.ylabel(f'{hps[idx]} (N)')
    plt.xlabel('Zenith (deg)')
    
plt.tight_layout()
plt.savefig('Evolution_hardpoint_forces_in.png')

## Plot specific iteration

In [ ]:
hps = ['HP1', 'HP2', 'HP3', 'HP4', 'HP5', 'HP6']
hpsm = ['measuredForce0', 'measuredForce1', 'measuredForce2', 'measuredForce3', 'measuredForce4','measuredForce5']

fig = plt.figure(figsize = (10,6))
for idx in range(6):
    plt.subplot(3,3,idx + 1)
    plt.plot(90 - elevations_hp_in_it7[2:], forces_hp_in_it7[hpsm[idx]][2:], '-', label = 'Balance forces')
    plt.title(f'{hps[idx]} - iteration 7')
    plt.ylabel(f'{hps[idx]} (N)')
    plt.xlabel('Zenith (deg)')

    
plt.tight_layout()
plt.savefig('Evolution_hardpoint_forces_in.png')

## Plot LUT Performance for the Joint Status Review

Want a plot showing (3x3 plot): 
* the first iteration, 
* the residuals when collecting the data for the second iteration
* Hardpoint forces after first iteration
* The sixth iteration 
* The residuals when collecting the data for the last iteration
* Hardpoint forces after sixth iteration
* The final iteration 
* The residuals of the final iteration 
* Hardpoint forces after last iteration

### Plot Data


In [ ]:
def find_associated_index(search_value):
    for fa in FATable:
        if fa.actuator_id == search_value:
            return fa.index
    return None

In [ ]:
# These actuators are in +X+Y, +X-Y, -X-Y, and -X,-Y
actuators = [122, 222, 322, 422]

fig, axs = plt.subplots(
    ncols=3,
    num="M1M3 LUT Performance for JSR", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 2),
    sharex="col",
)

fa_colors = cm.viridis(np.linspace(0, 0.9, 4))
er_colors = cm.Spectral(np.linspace(0, 0.9, 4))
hp_colors = cm.plasma(np.linspace(0, 0.9, 6))


for i, actuator in enumerate(actuators):
    idx = find_associated_index(actuator)
    axs[0].plot(90 - elevations_it0, forces_it0[zForce[idx]], '-', color=fa_colors[i], label=f"zForces #{actuator}")
    axs[1].plot(90 - elevations_it6, forces_it6[zForce[idx]], '-', color=fa_colors[i], label=f"zForces #{actuator}")
    axs[2].plot(90 - elevations_it7, forces_it7[zForce[idx]], '-', color=fa_colors[i], label=f"zForces #{actuator}")
    
axs[0].set_ylabel("Force Actuators\n Applied Balance\n Forces [N]")
axs[0].legend(bbox_to_anchor=(0.5, -0.4), loc="lower center", bbox_transform=fig.transFigure, ncol=4)
    
titles = ["Iteration 0", "Iteration 6", "Iteration 7"]
for ax, title in zip(axs, titles):
    ax.grid(":", alpha=0.5)
    ax.set_xlabel("Zenith Angle\n [deg]")
    ax.set_title(title)

# fig.suptitle("Evolution of the impact of the LUT\n on Force Actuators", y=1.1)
fig.tight_layout()

fig.savefig("jsr_lut_applied_forces.png", bbox_inches="tight")
plt.show()

In [ ]:
# These actuators are in +X+Y, +X-Y, -X-Y, and -X,-Y
fig, axs = plt.subplots(
    ncols=3,
    num="M1M3 LUT Performance for JSR - HardPoints", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 2),
)

hp_colors = cm.plasma(np.linspace(0, 0.9, 6))

# We know that there are a few outliers on this dataset. 
# We need to investigate. 
# But I'll remove some of them for now to improve the plot clarity.
_filter = (forces_hp_in_it6["measuredForce3"] < -400) & (forces_hp_in_it6["measuredForce0"] < -400)

forces_hp_in_it6_x = forces_hp_in_it6[_filter] 
elevations_hp_in_it6_x = elevations_hp_in_it6[_filter]

for idx in range(6):
    axs[0].plot(
        90 - elevations_hp_in_it0[2:], 
        forces_hp_in_it0[hpsm[idx]][2:], 
        '-', 
        color=hp_colors[idx], 
        label=f"Meas. Force HP{idx+1}"
    )
    
    axs[1].plot(
        90 - elevations_hp_in_it6_x[2:], 
        forces_hp_in_it6_x[hpsm[idx]][2:], 
        '-', 
        color=hp_colors[idx], 
        label=f"Meas. Force HP{idx+1}"
    )
    
    axs[2].plot(
        90 - elevations_hp_in_it7[2:], 
        forces_hp_in_it7[hpsm[idx]][2:], 
        '-', 
        color=hp_colors[idx], 
        label=f"Meas. Force HP{idx+1}"
    )
    
axs[0].set_ylabel("HP Measured Forces [N]")
axs[0].legend(
    bbox_to_anchor=(0.5, -0.4), 
    loc="lower center", 
    bbox_transform=fig.transFigure, 
    ncol=6
)

titles = ["Iteration 0", "Iteration 6", "Iteration 7"]
for ax, title in zip(axs, titles):
    ax.grid(":", alpha=0.5)
    ax.set_xlabel("Zenith Angle\n [deg]")
    ax.set_title(title)

fig.tight_layout()

fig.savefig("jsr_lut_HP.png", bbox_inches="tight")
plt.show()

In [ ]:
forces_hp_in_it6_x